In [ ]:
import cv2
import numpy as np
import datetime

def score_cleanliness(frame):
    # Focus only on lower 40% of frame (ground assumption)
    h = frame.shape[0]
    roi = frame[int(h*0.6):, :]

    # Convert to HSV for better color separation
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Create mask for "unusual" colors (simulate litter: reds, blues, bright)
    lower1 = np.array([0, 80, 60])
    upper1 = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower1, upper1)  # red

    lower2 = np.array([100, 100, 100])
    upper2 = np.array([140, 255, 255])
    mask2 = cv2.inRange(hsv, lower2, upper2)  # blue/plastic

    litter_mask = cv2.bitwise_or(mask1, mask2)

    # Find blobs of litter
    contours, _ = cv2.findContours(litter_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    litter_area = sum(cv2.contourArea(c) for c in contours)
    total_area = roi.shape[0] * roi.shape[1]
    litter_ratio = litter_area / total_area

    # Cleanliness scoring logic
    score = max(0, 100 - int(litter_ratio * 250))  # heavier penalty
    return score, contours, roi

def main():
    cap = cv2.VideoCapture(0)

    zone = "Zone A"
    last_cleaned = "2024-06-01"

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        score, litter_contours, roi = score_cleanliness(frame)

        offset_y = int(frame.shape[0] * 0.6)
        for cnt in litter_contours:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y + offset_y), (x + w, y + h + offset_y), (0, 0, 255), 2)

        # Display Score
        color = (0, 255, 0) if score > 75 else (0, 165, 255) if score > 50 else (0, 0, 255)
        cv2.putText(frame, f"Cleanliness Score: {score}%", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        cv2.putText(frame, f"Zone: {zone} | Last Cleaned: {last_cleaned}",
                    (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)

        cv2.imshow("🧹 Cleanliness Estimator (Press Q to Quit)", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [ ]:
import cv2
import mediapipe as mp
import pyautogui

# Initialize MediaPipe Hand Detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Define screen zones (OpenZone Concept)
zones = {
    "top_left": ((0, 0), (200, 200)),
    "top_right": ((440, 0), (640, 200)),
}

# Webcam capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmark in result.multi_hand_landmarks:
            for id, lm in enumerate(hand_landmark.landmark):
                cx, cy = int(lm.x * w), int(lm.y * h)

                # Tip of index finger
                if id == 8:
                    cv2.circle(frame, (cx, cy), 10, (255, 0, 255), -1)

                    # Zone Detection
                    for zone_name, ((x1, y1), (x2, y2)) in zones.items():
                        if x1 <= cx <= x2 and y1 <= cy <= y2:
                            cv2.putText(frame, f"TAP: {zone_name}", (x1, y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                            # Simulate mouse click if needed
                            if zone_name == "top_right":
                                pyautogui.click()
                else:
                    continue

            mp_draw.draw_landmarks(frame, hand_landmark, mp_hands.HAND_CONNECTIONS)

    # Draw Zones
    for zone_name, ((x1, y1), (x2, y2)) in zones.items():
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 1)
        cv2.putText(frame, zone_name, (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    cv2.imshow("Virtual Touchscreen", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import pyautogui

# Webcam capture
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Width
cap.set(4, 480)  # Height

# Initialize detector
detector = HandDetector(maxHands=1, detectionCon=0.8)

# Define screen zones (OpenZone-style virtual buttons)
zones = {
    "top_left": ((0, 0), (200, 200)),
    "top_right": ((440, 0), (640, 200)),
}

clicked_zone = None

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img)

    if hands:
        lmList = hands[0]["lmList"]  # 21 Landmark points
        index_finger_tip = lmList[8]  # Index finger tip
        x, y = index_finger_tip[0], index_finger_tip[1]

        cv2.circle(img, (x, y), 10, (255, 0, 255), -1)

        for zone_name, ((x1, y1), (x2, y2)) in zones.items():
            if x1 <= x <= x2 and y1 <= y <= y2:
                if clicked_zone != zone_name:
                    print(f"Clicked: {zone_name}")
                    pyautogui.click()
                    clicked_zone = zone_name
                cv2.putText(img, f"TAPPED: {zone_name}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                break
        else:
            clicked_zone = None

    # Draw zones
    for zone_name, ((x1, y1), (x2, y2)) in zones.items():
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 255), 1)
        cv2.putText(img, zone_name, (x1, y2 + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    cv2.imshow("Virtual Touchscreen", img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'mediapipe'

In [3]:
import cv2
import numpy as np
import pyautogui

# Setup webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Define OpenZone-style virtual buttons
zones = {
    "Top Left": ((0, 0), (200, 200)),
    "Top Right": ((440, 0), (640, 200)),
}

clicked_zone = None

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    # Convert to HSV for better color detection
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define green color range (you can adjust this if needed)
    lower_green = np.array([35, 60, 60])
    upper_green = np.array([85, 255, 255])

    # Create mask
    mask = cv2.inRange(hsv, lower_green, upper_green)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Process largest green object (your finger tip with green tape)
    if contours:
        max_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(max_contour) > 1000:
            x, y, w, h = cv2.boundingRect(max_contour)
            cx, cy = x + w // 2, y + h // 2

            # Draw center point
            cv2.circle(frame, (cx, cy), 10, (255, 0, 255), -1)

            # Check zones
            for zone_name, ((x1, y1), (x2, y2)) in zones.items():
                if x1 <= cx <= x2 and y1 <= cy <= y2:
                    if clicked_zone != zone_name:
                        print(f"Touch Detected in Zone: {zone_name}")
                        pyautogui.click()
                        clicked_zone = zone_name
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cv2.putText(frame, f"CLICK: {zone_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    break
            else:
                clicked_zone = None

    # Draw zones
    for zone_name, ((x1, y1), (x2, y2)) in zones.items():
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
        cv2.putText(frame, zone_name, (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    cv2.imshow("Touchscreen via Webcam", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import pyautogui

# Define Zones like virtual buttons
zones = {
    "top_left": ((0, 0), (200, 200)),
    "top_right": ((440, 0), (640, 200)),
}

clicked_zone = None

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    roi = frame.copy()

    # Convert to HSV and threshold skin color
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Blur and find contours
    mask = cv2.GaussianBlur(mask, (5, 5), 100)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if contours and len(contours) > 0:
        # Find largest contour (most likely hand)
        contour = max(contours, key=lambda x: cv2.contourArea(x))
        if cv2.contourArea(contour) > 1000:
            # Draw contour
            cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)

            # Find convex hull
            hull = cv2.convexHull(contour)
            cv2.drawContours(frame, [hull], -1, (255, 0, 0), 2)

            # Find fingertip (highest y)
            top = tuple(contour[contour[:, :, 1].argmin()][0])
            cv2.circle(frame, top, 10, (0, 0, 255), -1)

            x, y = top

            # Check if finger is in a zone
            for zone_name, ((x1, y1), (x2, y2)) in zones.items():
                if x1 <= x <= x2 and y1 <= y <= y2:
                    if clicked_zone != zone_name:
                        print(f"Clicked on: {zone_name}")
                        pyautogui.click()
                        clicked_zone = zone_name
                    cv2.putText(frame, f"TAP: {zone_name}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    break
            else:
                clicked_zone = None

    # Draw static zones
    for zone_name, ((x1, y1), (x2, y2)) in zones.items():
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 1)
        cv2.putText(frame, zone_name, (x1, y2 + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    cv2.imshow("Virtual Touch via Camera", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
Clicked on: top_left
